In [1]:
#导入需要的包
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import datetime
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from chinese_calendar import is_holiday

In [2]:
#从文件夹中读取csv格式文件并将其转换为DataFrame
def load_data(file_path, file_name):
	'''
    file_path : 文件路径名，即Data或temp

    file_name : 文件名，如：lai.csv
    '''
	df = pd.read_csv("../" + file_path + "/" + file_name, encoding='gbk')
	columns = df.columns
	df.fillna(df.mean(numeric_only=True), inplace=True)
	return df

In [3]:
#截取样本并根据列值去除异常样本
def cut_data(df, column_name, cut_index, abnormal_list):
	'''
    df : DataFrame格式数据

    column_name : 将要作为截取依据的列，如：要去除非用餐样本，可以将“商户名称”作为依据

    cut_index : 截取的下标，如：在lai.csv中一区数据从591开始

    abnormal_list : 异常样本列表
    '''
	cut_df = df[:cut_index]

	# 创建一个空的 DataFrame 用于存储筛选后的数据
	filtered_df = pd.DataFrame(columns=df.columns)

	for index, row in cut_df.iterrows():
		if row[column_name] in abnormal_list:
			continue
		else:
			# 将不满足条件的行添加到 filtered_df
			filtered_df = pd.concat([filtered_df, row.to_frame().T], ignore_index=True)
	return filtered_df

In [4]:
#将前后间隔不超过30分钟且刷卡地点相同的数据合并
def merge_data(df):
	'''
    df : DataFrame格式数据
    '''
	#创建一个空的DataFrame或列表来存储处理后的数据。
	processed_data = pd.DataFrame(columns=df.columns)

	df['时间'] = df['交易日期'] + ' ' + df['交易时间']
	df['时间'] = pd.to_datetime(df['时间'], format='%Y/%m/%d %H:%M')

	prev_row = None

	for index, row in df.iterrows():
		if prev_row is None:
			prev_row = row
		else:
			time_diff = prev_row['时间'] - row['时间']
			if (time_diff <= pd.Timedelta(minutes=30)) and (row['商户名称'] == prev_row['商户名称']) and (
					row['学工号'] == prev_row['学工号']):
				prev_row['交易额'] += row['交易额']
			else:
				processed_data = pd.concat([processed_data, prev_row.to_frame().T], ignore_index=True)
				prev_row = row

	# 处理最后一行数据
	processed_data = pd.concat([processed_data, prev_row.to_frame().T], ignore_index=True)
	return processed_data

In [5]:
#添加特征列，列值为时间的小时整点，用于与天气数据连接
def time2int(df):
	'''
    df : DataFrame格式数据
    '''
	# 将交易时间数据保存在一个名为"trade_times"的列表中
	trade_times = df['交易时间']

	processed_times = []

	for time_str in trade_times:
		# 将时间字符串转换为datetime对象
		dt = datetime.strptime(time_str, "%H:%M")
		# 如果分钟大于等于30，则小时进一位，并将分钟置为0
		if dt.minute >= 30:
			dt = dt.replace(hour=dt.hour + 1, minute=0)
		else:
			dt = dt.replace(minute=0)
		# 将处理后的时间添加到列表中
		processed_times.append(dt.strftime("%H:%M"))

	df['时间'] = processed_times
	return df

In [26]:
#将星期特征转换为数值，如 ：星期二 -> 2
def week2num(df, model = '星期'):
    '''
    df : DataFrame格式数据
    
    model : df的列名，默认为“星期”
    '''
    week = []

    for w in df[model]:
        if w == '星期一' or w == 'Monday':
            week.append(1)
        elif w == '星期二' or w == 'Tuesday':
            week.append(2)
        elif w == '星期三' or w == 'Wednesday':
            week.append(3)
        elif w == '星期四' or w == 'Thursday':
            week.append(4)
        elif w == '星期五' or w == 'Friday':
            week.append(5)
        elif w == '星期六' or w == 'Saturday':
            week.append(6)
        else:
            week.append(7)
            
    if model == '星期':
        df['星期'] = week
        return df
    else:
        return week

In [7]:
#将特殊天象数值化，若为空则为0，否则为1
def weather2num(df):
	'''
    df : DataFrame格式数据
    '''
	w_list = []
	weather = df['特殊天象'].values

	for w in weather:
		if str(w) == 'nan':
			w_list.append(0)
		else:
			w_list.append(1)
	df['特殊天象'] = w_list
	return df

In [8]:
#增添节假日特征
def add_holiday(df):
	'''
    df : DataFrame格式数据
    '''
	holiday = []

	for date in df['交易日期']:
		date = datetime.strptime(date, "%Y/%m/%d").date()
		if is_holiday(date):
			holiday.append(1)
		else:
			holiday.append(0)

	df['节假日'] = holiday
	return df

In [9]:
#增添时间戳特征
def add_timestamp(df):
	'''
    df : DataFrame格式数据
    '''
	df['时间戳'] = df['交易日期'] + ' ' + df['交易时间']
	df['时间戳'] = df['时间戳'].apply(lambda x: time.mktime(time.strptime(x, '%Y/%m/%d %H:%M')))
	return df

In [10]:
#将原数据与天气数据结合起来
def combine_climate(df, climate):
	'''
    df : DataFrame格式数据

    climate : DataFrame格式数据，包含天气信息
    '''
	#将不足五位的天气数据用0补足，如7:00 -> 07:00
	climate['时间'] = climate['时间'].str.zfill(5)

	df['日期小时'] = df['交易日期'] + ' ' + df['时间'].str[:2]
	climate['日期小时'] = climate['date'] + ' ' + climate['时间'].str[:2]

	merged_df = pd.merge(df, climate, on='日期小时')
	merged_df = merged_df.drop(['学工号', '姓名', '日期小时', 'date', '时间_x', '时间_y', '重要天象'], axis=1)
	return merged_df

In [11]:
#筛选合格的样本，如去除消费值过低（可能是买水等因素）的数据
def select_data(df):
	'''
    df : DataFrame格式数据

    '''
	# 创建一个时间对象表示 10:00，用于区分上午和其他时段（早餐消费可能偏低）
	threshold_time = pd.to_datetime('10:00', format='%H:%M')

	# 创建一个空的 DataFrame 用于存储筛选后的数据
	filtered_df = pd.DataFrame(columns=df.columns)

	# 迭代 DataFrame 行
	for index, row in df.iterrows():
		if float(row['交易额']) <= 5 and pd.to_datetime(row['交易时间'], format='%H:%M') > threshold_time:
			# 跳过满足条件的行
			continue
		elif float(row['交易额']) <= 2 and pd.to_datetime(row['交易时间'], format='%H:%M') < threshold_time:
			# 跳过满足条件的行
			continue
		else:
			# 将不满足条件的行添加到 filtered_df
			filtered_df = pd.concat([filtered_df, row.to_frame().T], ignore_index=True)
	return filtered_df

In [29]:
# 根据课表信息生成个人数据库
def creat_database(file_path):
    original_personal_class1 = load_data(file_path,'transform_2022秋_lai.csv')
    original_personal_class2 = load_data(file_path,'transform_2022夏_lai.csv')
    original_personal_class3 = load_data(file_path,'transform_2023春_lai.csv')
    original_personal_class=pd.concat([original_personal_class1,original_personal_class2],copy=True)
    original_personal_class=pd.concat([original_personal_class,original_personal_class3],copy=True)
    original_personal_class['Start Date'] = pd.to_datetime(
        original_personal_class['Start Date'])
    original_personal_class['Start Date'] = original_personal_class['Start Date'].dt.strftime(
        '%Y/%m/%d')
    original_personal_class['End Date'] = pd.to_datetime(
        original_personal_class['End Date'])
    original_personal_class['End Date'] = original_personal_class['End Date'].dt.strftime(
        '%Y/%m/%d')
    original_personal_class['Weekday'] = week2num(original_personal_class, model = 'Weekday')
    courseType = ['Morning', 'Afternoon', 'Evening']
    dataList = pd.date_range(
        start='20220713', end='20230616').strftime('%Y/%m/%d').tolist()
    personal_class = pd.DataFrame(0, index=courseType, columns=dataList)

    for index, row in original_personal_class.iterrows():
        column = pd.date_range(
        start=row['Start Date'], end=row['End Date'], freq='W-MON')
        
        if row['Start Time'] == '08:00':
            personal_class.loc['Morning',
                               df.columns.isin(column)] = 1
        elif row['End Time'] == '11:45':
            personal_class.loc['Afternoon',
                               df.columns.isin(column)] = 1
        elif row['End Time'] == '17:30':
            personal_class.loc['Evening',
                               df.columns.isin(column)] = 1

    return personal_class


In [13]:
# 判断当前交易行为与课程间的关系
def combine_class(df, personal_class):
    # 类似one-hot encode，采用二进制表示法，每一位表示一个餐点前有没有课
    temp=df
    temp['交易日期'] = pd.to_datetime(temp['交易日期'])
    temp['交易日期'] = temp['交易日期'].dt.strftime('%Y/%m/%d')
    list_1 = []
    list_2 = []
    list_3 = []
    for index, row in temp.iterrows():
        list_1.append(personal_class[row['交易日期']]['Morning'])
        list_2.append(personal_class[row['交易日期']]['Afternoon'])
        list_3.append(personal_class[row['交易日期']]['Evening'])
    temp['8点上课']=list_1
    temp['午饭有课']=list_2
    temp['晚饭有课']=list_3
    return temp


In [14]:
#保存数据为csv文件
def save_data(df, file_path, file_name):
	'''
    df : DataFrame格式数据

    file_path : 文件路径名，即Data或temp

    file_name : 文件名，如：lai.csv
    '''

	df.to_csv('../' + file_path + '/' + file_name, sep=',', encoding='gbk', index=False)

In [30]:
#通过更改文件名，处理不同组数据
df = load_data('Data', 'lai.csv')
climate = load_data('Data', 'climate.csv')

abnormal_merchant = np.array(['淘乐学苑水果', '学苑食堂霞姐饮品店', '中央红小月亮门店5', '中央红小月亮门店2' \
								 , '中央红小月亮门店4', '中央红小月亮门店6', '中央红小月亮门店3', '中央红小月亮正心楼' \
								 , '中央红-水果', '哈尔滨市南岗区淘乐水果捞店', '中央红小月亮门店1', '中央红-药店' \
								 , '深澜网费对接', '中央红-辣货', '紫丁香餐吧酒水（聚鑫食品）', '回味斋一餐厅酒水组' \
								 , '美芝林快客', '灌制间'])
df = cut_data(df, '商户名称', 590, abnormal_merchant)
df = merge_data(df)
df = time2int(df)
df = add_holiday(df)
df = combine_climate(df, climate)
df = add_timestamp(df)
df = week2num(df)
df = weather2num(df)
df = select_data(df)
db_c=creat_database('temp')
df = combine_class(df,db_c)
df

DatetimeIndex(['2022-08-22'], dtype='datetime64[ns]', freq='W-MON')


IndexError: Boolean index has wrong length: 13 instead of 339

In [ ]:
#保存到temp目录下
save_data(df, 'temp', '赖（final）.csv')
save_data(db_c, 'temp', 'db_c_lai.csv')
